
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: aplicación de las SVM a un problema de regresión</h3></font>
***

Este *notebook* contiene la aplicación de las SVM a un problema de regresión. Concretamente, usaremos el problema de tasación de casas en Ames (Iowa, USA) desde 2006 hasta 2010. 

El conjunto de entrenamiento contiene 1460 observaciones y un gran número de variables explicativas (23 nominales, 23 ordinales, 14 discretas y 20 continuas). Para más detalle, puede acudirse al archivo de texto data_description.txt que se encuentra en la misma carpeta.

In [1]:
## -------------------------------- ##
## Librerías
## -------------------------------- ##

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline

# ------------------------------------------------------------------------------
# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

Antes de empezar, vamos a definir una función para obtener estadísticas del modelo de regresión.

In [2]:
startbold = '\033[1m'
endbold = '\033[0m'

def estadisticas_modelo(model, x_train, y_train, x_test, y_test):
    
    ## Métricas para el conjunto de train
    cadena = "Mean absolute error (MAE) en training: "+startbold+"{:.2f}"+endbold
    print(cadena.format(np.mean(np.abs((model.predict(x_train) - y_train)))))
    cadena = "Maximum absolute error en training: "+startbold+"{:.2f}"+endbold
    print(cadena.format(np.max(np.abs((model.predict(x_train) - y_train)))))
    cadena = "R2 en training (1: perfecto): "+startbold+"{:.2f}"+endbold
    print(cadena.format(model.score(x_train, y_train)))
    print()
    
    ## Métricas para el conjunto de test
    cadena = "Mean absolute error (MAE) en test: "+startbold+"{:.2f}"+endbold
    print(cadena.format(np.mean(np.abs((model.predict(x_test) - y_test)))))
    cadena = "Maximum absolute error en test: "+startbold+"{:.2f}"+endbold
    print(cadena.format(np.max(np.abs((model.predict(x_test) - y_test)))))
    cadena = "R2 en test (1: perfecto): "+startbold+"{:.2f}"+endbold
    print(cadena.format(model.score(x_test, y_test)))
    print()

## Lectura de los datos

In [ ]:
df = pd.read_csv('./data/ames_housing.csv')

print(f"Dimensiones: {df.shape}")
df.head()

## Análisis exploratorio (EDA)

In [ ]:
df.describe().T

## Existen valores ausentes
## Variabilidad en los datos y escalas muy diferentes -> normalizar/estandarizar!

Como la variable a predecir es *SalePrice*, vamos a extraer algunas estadísticas para ver cómo se distribuye.

In [ ]:
print("Ahora sacaremos algunas estadísticas de la base de datos")
print('\nLa mediana de "SalePrice" es', df['SalePrice'].median(axis = 0), "\n")
df[['SalePrice']].describe()

## El target no contiene valores ausentes

In [ ]:
plt.figure(figsize=(10,4))
ax = sns.distplot(df['SalePrice'], kde = True)
plt.show()
print("Skewness: %f" % df['SalePrice'].skew())
print("Kurtosis: %f" % df['SalePrice'].kurt())

## Asimétrica positiva, contiene valores extremos positivos

Vemos cómo correlan las variables explicativas con la variable a predecir.

In [ ]:
corr = df.select_dtypes(include = ['float64', 'int64']).iloc[:, 1:].corr()
plt.figure(figsize=(12, 12))
sns.heatmap(corr, vmin=-1, vmax=1, square=True, cmap=sns.color_palette("RdBu_r", 21))
plt.show()

In [ ]:
sns.clustermap(corr, vmin=-1, vmax=1, cmap=sns.color_palette("RdBu_r", 21), yticklabels = True, xticklabels = True)
plt.show()

## Vemos que existen variables muy correlacionadas con la variable a predecir y otras no tienen prácticamente relación lineal

In [ ]:
## Para simplificar, seleccionamos estas columnas

sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df[cols], size = 2.5, kind='reg',
             plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}})
plt.show()

Como OverallQual es la más correlacionada con la variable a predecir, vamos a enfrentarlas.

In [ ]:
## Enfrentamos SalePrice VS OverallQual

data = pd.concat([df['OverallQual'], df['SalePrice']], axis=1)
plt.figure(figsize=(8, 3))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=data)
fig.set_xlabel('OverallQual', fontsize=16)
fig.set_ylabel('SalePrice', fontsize=16);

## A mayor calidad, el precio de venta es mayor

In [ ]:
## Atributos categóricos
print(df.select_dtypes(include=['object']).columns.values)

In [ ]:
## Atributos numéricos
print(df.select_dtypes(exclude=['object']).columns.values)

## Regresión lineal

Primeramente, vamos a hacer una regresión lineal para tener un *baseline*.

#### Partición del dataset

In [15]:
## División en train y test
from sklearn.model_selection import train_test_split

testsize = 0.2
df_train, df_test = train_test_split(df, test_size=testsize, random_state=3)

#### Ranking de atributos

Con el objetivo de seleccionar variables que aporten calidad en la predicción del modelo y, además, sin valores ausentes, calculamos correlaciones lineales con la variable a predecir.

In [ ]:
print("\nRanking de atributos por el valor absoluto de su")
print("correlación con el target (SalePrice):\n")
pd.DataFrame((df_train.corr()["SalePrice"].drop("SalePrice").abs()).sort_values(ascending=False))

In [ ]:
## Selección de atributos

selected_attributes = ['OverallQual', 'GrLivArea', 'GarageCars',
                       'TotalBsmtSF', 'FullBath', 'YearBuilt']

x_train = df_train.loc[:,selected_attributes].values
y_train = df_train['SalePrice'].values
x_test = df_test.loc[:,selected_attributes].values
y_test = df_test['SalePrice'].values
y = df['SalePrice'].values
atributos = selected_attributes

print(f"Dimensiones: {x_train.shape}")
print(f"Dimensiones: {y_train.shape}")

Entrenamos el modelo de regresión lineal.

In [ ]:
regression = LinearRegression()
regression.fit(x_train, y_train)

print(startbold+"\n- Regresión lineal estándar -"+endbold)

print(startbold+"\nIntercept:"+endbold)
print(regression.intercept_)
print(startbold+"\nCoeficientes:"+endbold)
print(regression.coef_)
print()

Mirando el rendimiento en el conjunto de test se tiene que,

* Existe sobreajuste.
* En media, nos equivocamos unos 25.500 $ (MAE).

In [ ]:
y_train_predicted = regression.predict(x_train)
y_test_predicted = regression.predict(x_test)

estadisticas_modelo(regression, x_train, y_train, x_test, y_test)

Dibujando valores reales frente a predichos, se observa que hay un sesgo en precios elevados: el modelo subestima estos valores.

In [ ]:
y_train_predicted = regression.predict(x_train)
y_test_predicted = regression.predict(x_test)

font_size = 14
plt.figure(figsize=(15,5))
ax1 = plt.subplot(1,2,1)
plt.plot(y_train, y_train_predicted, 'b.', zorder=1)
lims = [y.min(), y.max()]
plt.plot(lims, lims, 'k-', zorder=1)
ax1.set_xlim(lims)
ax1.set_ylim(lims)
plt.xlabel('y real', fontsize=font_size)
plt.ylabel('y predicho', fontsize=font_size)
plt.title('conjunto de training', fontsize=font_size)
plt.grid(True)
plt.axis('equal')

ax2 = plt.subplot(1, 2, 2)
plt.plot(y_test, y_test_predicted, 'g.', zorder=1)
plt.plot(lims, lims, 'k-', zorder=1)
ax2.set_xlim(lims)
ax2.set_ylim(lims)
plt.xlabel('y real', fontsize=font_size)
plt.ylabel('y predicho', fontsize=font_size)
plt.title('conjunto de test', fontsize=font_size)
plt.grid(True)
plt.axis('equal');

Al observar un sesgo en precios más elevados, analizamos los residuos (diferencia entre valores predichos y reales) para verificarlo, siendo efectivamente así.

In [ ]:
plt.scatter(y_train_predicted,  y_train_predicted - y_train,
            c='blue', marker='o', label='training', alpha=0.5)
plt.scatter(y_test_predicted,  y_test_predicted - y_test,
            c='lightgreen', marker='s', label='test', alpha=0.5)
plt.xlabel('Valores predichos')
plt.ylabel('Residuos')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=y.min(), xmax=y.max(), lw=2, color='red')
plt.xlim([y.min(), y.max()])
plt.tight_layout();

In [ ]:
## Estadísticas de los residuos en los conjuntos de training y test

plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
sns.distplot(y_train-y_train_predicted, kde = True)
aux = y_train-y_train_predicted
lims = [np.mean(aux)-3*np.std(aux), np.mean(aux)+3*np.std(aux)]
plt.xlim(lims)
plt.title('conjunto de training')
plt.xlabel('residuos')
plt.subplot(1,2,2)
sns.distplot(y_test-y_test_predicted, kde = True)
plt.xlim(lims)
plt.title('conjunto de test')
plt.xlabel('residuos')
plt.tight_layout();

## Máquinas de Soporte Vectorial para Regresión (SVR)

Ahora, vamos a entrenar un SVR. En este caso, vamos a estandarizar las variables predictoras. Cabe resaltar, que la variable a predecir también tiene que ser estandarizada, ya que sino estaríamos entrenando con un rango de valores muy pequeño para predecir un rango amplio y elevado de valores.

Para ello, vamos a tener en cuenta dos casos: (1) sin estandarizar la variable a predecir (2) sí estandarizándola. Esto lo hacemos así para **poner de manifiesto la importancia de la también estandarización de la variable a predecir.**

In [23]:
## Otras librerías que necesitamos

from sklearn.svm import SVR
import sklearn.preprocessing as skpp

### Sin estandarización de la variable a predecir

In [ ]:
## Estandarizamos los datos de entrada al modelo
scalerX = skpp.StandardScaler()
scalerX.fit(x_train)

## Aplicamos la estandarización a los datos
x_train_scal = scalerX.transform(x_train)
x_test_scal = scalerX.transform(x_test)

## Construcción del modelo:
svr = SVR(kernel='rbf', epsilon=100000, C=100)
svr.fit(x_train_scal, y_train)

## Rendimiento del modelo
print(startbold+"\n- Resultados del SVR -\n"+endbold)

estadisticas_modelo(svr, x_train_scal, y_train, x_test_scal, y_test)

Se observa que el coeficiente de determinación (R squared) es muy pobre, llegando a ser incluso negativo.

Probamos a subir el factor de complejidad, ya que hará que el modelo sea más complejo.

In [ ]:
## Estandarizamos los datos de entrada al modelo
scalerX = skpp.StandardScaler()
scalerX.fit(x_train)

## Aplicamos la estandarización a los datos
x_train_scal = scalerX.transform(x_train)
x_test_scal = scalerX.transform(x_test)

## Construcción del modelo:
svr = SVR(kernel='rbf', epsilon=100000, C=100000)
svr.fit(x_train_scal, y_train)

## Rendimiento del modelo
print(startbold+"\n- Resultados del SVR -\n"+endbold)

estadisticas_modelo(svr, x_train_scal, y_train, x_test_scal, y_test)

Subiendo el factor de complejidad, sí logramos mejorar las métricas, pero no es lo más efectivo al construir un modelo que no generalice bien.

### Con estandarización de la variable a predecir

In [ ]:
## Estandarizamos los datos de entrada al modelo
scalerX = skpp.StandardScaler()
scalerX.fit(x_train)

## Aplicamos la estandarización a los datos (explicativas + target)
x_train_scal = scalerX.transform(x_train)
x_test_scal = scalerX.transform(x_test)

mean_y = y_train.mean()
std_y  = y_train.std()
y_train_scal = (y_train-mean_y)/std_y
y_test_scal  = (y_test-mean_y)/std_y

## Construcción del modelo
svr = SVR(kernel='rbf', epsilon=0.1, C=1)
svr.fit(x_train_scal, y_train_scal)

## Rendimiento del modelo
print(startbold+"\n- Resultados del SVR -\n"+endbold)

estadisticas_modelo(svr, x_train_scal, y_train_scal, x_test_scal, y_test_scal)

Mejoramos muchísimo el rendimiento del modelo y, además, sin tener que aumentar el factor de complejidad, lo que nos hace tener un modelo general y que, además, presenta buen rendimiento.

### Predicción

Cabe tener en cuenta que **el modelo ha sido entrenado con precios estandarizados.** Por lo tanto, **si queremos predecir tenemos que deshacer la transformación hecha sobre la variable predictora.** Recordar que la transformación que se ha hecho es restar la media y dividir por la desviación típica. Luego, para deshacer esta transformación tenemos que multiplicar por la desviación típica y sumar la media.

In [27]:
## Deshacemos la transformación en train
y_train_predicted = mean_y + std_y*svr.predict(x_train_scal)

## Deshacemos la transformación en test
y_test_predicted  = mean_y + std_y*svr.predict(x_test_scal)

In [ ]:
## Visualizamos 

font_size = 14
plt.figure(figsize=(15,5))
ax1 = plt.subplot(1,2,1)
plt.plot(y_train, y_train_predicted, 'b.', zorder=1)
lims = [y.min(), y.max()]
plt.plot(lims, lims, 'k-', zorder=1)
ax1.set_xlim(lims)
ax1.set_ylim(lims)
plt.xlabel('y real', fontsize=font_size)
plt.ylabel('y predicho', fontsize=font_size)
plt.title('conjunto de training', fontsize=font_size)
plt.grid(True)
plt.axis('equal')

ax2 = plt.subplot(1, 2, 2)
plt.plot(y_test, y_test_predicted, 'g.', zorder=1)
plt.plot(lims, lims, 'k-', zorder=1)
ax2.set_xlim(lims)
ax2.set_ylim(lims)
plt.xlabel('y real', fontsize=font_size)
plt.ylabel('y predicho', fontsize=font_size)
plt.title('conjunto de test', fontsize=font_size)
plt.grid(True)
plt.axis('equal');